In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import font, filedialog, messagebox
from tkinter import filedialog
from tkinter.messagebox import *
import json
from metricsyso import *
import pandas as pd
import time
from tqdm import tqdm

class MyApp():
    def __init__(self,windows):
        self.windows = windows
        windows.title("My App")
        self.conf_df = None
        self.df_m =None 
        self.df_w =None
        self.dataframe_options = ["Confusion matrix","IoU_Rec_Prec_Acc_F1_score","Weighted IoU_Rec_Prec_Acc_F1_score"]
        
        # configure grid layout 
        windows.rowconfigure(0, weight=1)
        windows.columnconfigure(0,weight=1)
        
        for i in range(15):
            for j in range(100):
                 windows.rowconfigure(i, weight=1)
                 windows.columnconfigure(j, weight=1)
             
        # frame for getting ground truth file
        self.gtruth_file_entry = Entry(windows, width=50)
        self.gtruth_file_entry.grid(row=1,column=1, columnspan=2, pady=15)
        self.gtruth_file_btn = tk.Button(windows, text="Load Ground Truth File", command=lambda: self.gtruth_file_entry.insert(tk.END, self.browse_file("Select Ground Truth File")))
        self.gtruth_file_btn.grid(row=0,column=1, columnspan=2, pady=15)
        
        # fram e for getting prediction file
        self.pred_file_entry = Entry(windows, width=50)
        self.pred_file_entry.grid(row=3, column=1, columnspan=2, pady=5)
        self.pred_file_btn = tk.Button(windows, text="Load Prediction File", command=lambda: self.pred_file_entry.insert(tk.END, self.browse_file("Select Prediction File")))
        self.pred_file_btn.grid(row=2, column=1, columnspan=2, pady=5)
        
        # frame for geting entry for json file
        self.json_file_entry = Entry(windows, width=50)
        self.json_file_entry.grid(row=5, column=1, columnspan=2, pady=5)
        self.json_file_btn = tk.Button(windows, text="Load JSON File", command=lambda: self.json_file_entry.insert(tk.END, self.browse_file("Select JSON File")))
        self.json_file_btn.grid(row=4, column=1, columnspan=2, pady=5) 
        # Displaying buttons for metrics
        # dispalyfram
        """self.disframe=Frame(windows, borderwidth=2,bg='white', bd=3, cursor='hand2', height=50, 
                              highlightcolor='blue', highlightthickness=2, highlightbackground='black', 
                              relief=tk.GROOVE)  
        self.disframe.grid(row=3, column=0)"""
         # Creat a canvas
        self.canvas = Canvas(windows, borderwidth=1,bg='white', bd=2, cursor='hand2', height=10, 
                              highlightcolor='blue', highlightthickness=2, highlightbackground='black', 
                              relief=GROOVE)
        self.canvas.grid(row=7,column=0, sticky="NS")
        self.label=Label(self.canvas, text="Display metrics",bg='grey',fg='white', relief=RAISED, font=("Arial", 16, "bold"),anchor='center')
        self.label.grid(row=0, column=0, columnspan=2)
        
        #self.button1 = Button(windows,text="Display confusion matrix", command =self.display_confmatrix).place(x=60, y=200,height = 40, width = 180)
        self.button1 = Button(self.canvas, text="Confusion matrix", command =self.display_confmatrix)
        self.button2 = Button(self.canvas, text="IoU_Rec_Prec_Acc_F1_score", command =self.display_df_metrics)
        self.button3 = Button(self.canvas, text="weighted metrics", command =self.display_weightetd_metrics)
        self.buttonclose = Button (windows, text ="Exit", width=3, command = self.close_window)
        self.save_button = tk.Button(windows, text="Save to File", command=self.save_frame)
        
        #pack in grid
        self.button1.grid(row=4,column=0)
        self.button2.grid(row=5,column=0)
        self.button3.grid(row=6,column=0)
        self.save_button.grid(row=9, column=2, columnspan=2, pady=10)
        self.buttonclose.grid(row=9, column=3, columnspan=6, pady=10) 
        
        ## save option
        self.dataframe_var =  StringVar(windows)
        self.dataframe_var.set("Chose frame to save")
        self.dataframe_menu = OptionMenu(windows,self.dataframe_var, *self.dataframe_options)
        self.dataframe_menu.grid(row=9, column=1, columnspan=1)
        
       
        # Create a frame for displaying
        self.frame = tk.Frame(windows, borderwidth=2,bg='white', bd=3, cursor='hand2', height=50, 
                              highlightcolor='blue', highlightthickness=2, highlightbackground='black', 
                              relief=tk.RAISED)
        self.frame.grid_propagate(True)
        self.frame.grid(row=7,column=1, columnspan=80, padx=5,pady=5, sticky='NSEW')
       
        
        """ # Creat a canvas
        self.canvas = Canvas(self.frame)
        self.canvas.pack(fill=BOTH, expand=True)
        
        # Iiner frame to kepp treeview
        self.inner_frame = Frame(self.canvas)
        self.canvas.create_window((0,0), window=self.inner_frame, anchor="nw")
        self.inner_frame.bind("<Configure>",self.frame_configure)"""
        
        self.default_dict = default_dict = {"1":"default","2":"ground","5": "vegetation", 
                "6":"building","7": "wall", 
                "196": "structure","211":"cable"
                }
        #Label(self.frame, bg="white").pack(padx=10, pady=10)
        #self.button = Button(windows,text="Display confsusion matrix", command =self.compute_metrics).place(x=60, y=45,height = 50, width = 180)
        
        # Create buttons to compute metrics
        self.compute_button = tk.Button(windows, text="Compute Metrics",command=lambda: self.compute_metrics(self.gtruth_file_entry.get(),self.pred_file_entry.get(),self.json_file_entry.get()))
        self.compute_button.grid(row=0, column=0)
        # place a progressbar
        self.pb = ttk.Progressbar(windows, orient='horizontal',    mode='indeterminate',    length=280)
        self.pb.grid(row=1, column=0)
        

    
        # Create a treeview in the frame for dataframe pandas
        
        self.df_tree = ttk.Treeview(self.frame)
        #self.df_tree.grid(row=0,column=0, sticky="NSEW")
        self.df_tree.pack(fill=BOTH, expand=True)
        
        # ScrollBar for the frame an d treeview
        
    """# horizontal scrollbar
        self.horscrolbar = Scrollbar(self.frame, orient='horizontal', command= self.frame.xview) 
        #self.horscrolbar.grid(row=1, column=0,sticky="EW")
        self.horscrolbar.pack(side=BOTTOM)
        
        # vertical scrollbar
        self.verscrolbar =Scrollbar(self.frame, orient='vertical',command= self.frame.yview)
        self.verscrolbar.grid(row=0, column=1, sticky="NS")
        #self.verscrolbar.pack(side=RIGHT)
        #self.df_tree.tag_configure('ttk', background='yellow'
        #self.df_tree.configure(xscrollcommand= self.horscrolbar.set,yscrollcommand= self.verscrolbar.set)
        self.frame.configure(xscrollcommand= self.horscrolbar.set,yscrollcommand= self.verscrolbar.set)"""
        
        
        # update inner_framme to canvas
        
    
    """def frame_configure(self, event):
        self.canvas.configure(scrollregion=self.canvas.bbox("all"))
        self.canvas.itemconfig(self.canvas.find_all()[0],width=self.canvas.winfo_width(), height= self.canvas.winfo_height())"""
    
        #self.status_label = tk.Label(windows, text="", padx=20, pady=10)
        #self.status_label.pack()
        
        
        
        #self.text_widget = tk.Text(windows, wrap=tk.WORD)
        #self.text_widget.pack(padx=20, pady=20, fill="both", expand=True)
        
    """self.hscrolbar = Scrollbar(self.frame, orient='horizontal')
        self.hscrolbar.pack(side=BOTTOM, fill = X)
        self.vscrolbar = Scrollbar(self.frame, orient='vertical')
        self.vscrolbar.pack(side = RIGHT, fill = Y)"""
        
        
        


    def browse_file(self,title):
        filepath = filedialog.askopenfilename(title=title, filetypes=(("Laz or Las files", "*.laz *.las"),
                                                                    ("All files", "*.*") ,
                                                                    ("Json files","*.json")))
        return filepath


    def tree_display(self,df):
            self.df_tree.delete(*self.df_tree.get_children())
            
            #df_list=list(df.columns.values)
            df_with_index = df.reset_index()
            df_rset=df_with_index.to_numpy().tolist()
            #df['index'] = df.index
            
            #self.df_tree=ttk.Treeview(self.frame,columns=df_list)
            #self.df_tree.pack()                                      
            """for i in df_list:
                self.df_tree.column(i,width=100,anchor='c')
                self.df_tree.heading(i,text=i)"""
                
            columns= list(df_with_index.columns[:])
            #columns1 =np.concatenate(df.index.name,columns)
            self.df_tree["columns"] = columns
            self.df_tree ["show"] = "headings"
            for column in columns :
                self.df_tree.heading (column , text= column)
                self.df_tree.column(column, width= 50, anchor = 'c')
                
            for dt in df_rset:
                v=[r for r in dt]
                self.df_tree.insert('','end', values=v)
    

    def compute_metrics(self,gtruth_file,pred_file,json_file):
        
        #gtruth_file =self.gtruth_file_entry.get()
        #pred_file = self.pred_file_entry.get()
        #json_file = self.json_file_entry.get()
        
        
        # Reset pb bar
        self.pb['value'] = 0
        self.windows.update_idletasks()
        
        # step 1: Load Json file
        op = JsonFile(json_file)
        conv_dict = op.convdict()
        conv_class = op.convclass()
        truth_class = op.conv_to_Truclass(conv_dict, conv_class)
        self.pb['value'] = 20
        self.windows.update_idletasks() 

        # Step 1 : read gtruth file and  pred file
        gtruth = read_file_class(gtruth_file)
        prediction = read_file_class(pred_file)
        count_pred = num_points(prediction)
        count_gtruth = num_points(gtruth)
        self.pb['value'] = 40
        self.windows.update_idletasks()
        
        # step 3 : convert gtruth classes
        convertor = truth_class
        gtruth_conv = [convertor.get(str(k)) if convertor.get(str(k)) is not None else k for k in count_gtruth]
        self.pb['value'] = 60
        self.windows.update_idletasks()

        # step 4 : conpute confusion matrix
        input_pred = np.array(count_pred)
        input_gtruth = np.array(gtruth_conv)
        unique_classes =  np.unique(np.concatenate((input_gtruth, input_pred )))
        self.clas_name = [str(name) for name in tqdm(unique_classes)]
        self.conf_df= compute_confusion_matrix(input_gtruth, input_pred)
        self.conf_df.rename(columns = self.default_dict, index = self.default_dict, inplace = True)
        self.pb['value'] = 80
        self.windows.update_idletasks()
        
        # Step 5 compute metrics
        m = Metrics(input_gtruth, input_pred)
        metrics_fr = m.compute_metrics()
        self.df_m =m.metrics_frame(metrics_fr)
        self.df_m.rename(columns = self.default_dict, index = self.default_dict, inplace = True)
        self.df_w = m.weighted_metrics()
        self.pb['value'] = 100
        self.windows.update_idletasks()
        messagebox.showinfo("Succesfully computed")

    

    # displayinf data frame witrh a treeView
    
    def display_confmatrix(self):
        self.tree_display(self.conf_df)

    def display_df_metrics(self):
        self.tree_display(self.df_m)

    def display_weightetd_metrics(self):
        self.tree_display(self.df_w)
                    

    def save_to_file(self,df, sheet_name):
        files = [('All Files', '*.*'),  
             ('Text Document', '*.txt'),
             ('Excel files', '*.xlsx')] 
        file_path = filedialog.asksaveasfilename(filetypes = files, defaultextension ='.xlsx')
        if file_path:          
            try:
                with pd.ExcelWriter(file_path) as writer:
                    df.to_excel(writer, sheet_name= sheet_name, index=True)
                    messagebox.showinfo("Succes", f"Data saved to{file_path}")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to save data:{str(e)}")
        else:
            messagebox.showwarning("Canceled","Saving canceled")
               
    def save_frame(self):
        selected_dataframe = self.dataframe_var.get()
        if selected_dataframe == "Confusion matrix":
            self.save_to_file(self.conf_df, "Confusion_Matrix")
        elif selected_dataframe == "IoU_Rec_Prec_Acc_F1_score":
            self.save_to_file(self.df_m, "IoU_Rec_Prec_Acc_F1_score")
        elif selected_dataframe == "Weighted IoU_Rec_Prec_Acc_F1_score":
            self.save_to_file(self.df_w, "Weighetd IoU_Rec_Prec_Acc_F1_score")
    
                
    """def save_frame(self):
        selected_dataframe = dataframe_var.get()
        if self.tree_display(self.conf_df):
            self.save_to_file(self.conf_df, "Confusion_matrix")
        
        elif self.tree_display(self.df_m):
            self.save_to_file(self.df_m, "Metrics_Frame")
            
        elif self.tree_display(self.df_w):
            self.save_to_file(self.df_w, "Weighted_Metrics")"""
            
    
    """def save_frame(self):
        if self.tree_display(self.conf_df):
            if self.file_path:
                try:
                    with pd.ExcelWriter(self.file_path.name) as writer:
                        if hasattr(self, 'conf_df') and isinstance(self.conf_df,pd.DataFrame):
                            self.conf_df.to_excel(writer,sheet_name= "Confusion_Matrix", index=True)
                            messagebox.showinfo("Succes", f"Data saved to {self.file_path.name}")
                except Exception as e:
                    messagebox.showerror("Error", f"Failed to save data:{str(e)}")
        
        elif self.tree_display(self.df_m):
            if self.file_path:
                try:
                    with pd.ExcelWriter(self.file_path.name) as writer:
                        if hasattr(self, 'df_m') and isinstance(self.df_m,pd.DataFrame):
                            self.df_m.to_excel(writer, sheet_name= "DataFrame_Metrics", index=True)
                            messagebox.showinfo("Succes", f"Data saved to {self.file_path.name}")
                except Exception as e:
                    messagebox.showerror("Error", f"Failed to save data:{str(e)}")
        
        elif self.tree_display(self.df_w):
            if self.file_path:
                try:
                    with pd.ExcelWriter(self.file_path.name) as writer:
                        if hasattr(self, 'df_w') and isinstance(self.df_w,pd.DataFrame):
                            self.df_w.to_excel(writer,sheet_name= "Weighted_Metrics", index=True)
                            messagebox.showinfo("Succes", f"Data saved to {self.file_path.name}")
                except Exception as e:
                    messagebox.showerror("Error", f"Failed to save data:{str(e)}")"""
        
        
                
    def close_window(self):
        self.windows.destroy()
        
        
windows =Tk()
my_gui = MyApp(windows)
windows.geometry('900x600')
#askyesnocancel("askyesnocancel", "Yes or No or Cancel")
windows.mainloop()

100%|██████████| 25/25 [00:00<?, ?it/s]


len ground_truth 113550333
len pred 4994922
min length4994922


100%|██████████| 4994922/4994922 [00:18<00:00, 263601.38it/s]
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\ysouley\AppData\Local\Temp\ipykernel_23924\1795213089.py", line 110, in <lambda>
    self.compute_button = tk.Button(windows, text="Compute Metrics",command=lambda: self.compute_metrics(self.gtruth_file_entry.get(),self.pred_file_entry.get(),self.json_file_entry.get()))
                                                                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ysouley\AppData\Local\Temp\ipykernel_23924\1795213089.py", line 243, in compute_metrics
    metrics_fr = m.compute_metrics()
                 ^^^^^^^^^^^^^^^^^^^
  File "m:\#Utilisateurs\YSO\YSO_scripts\metric

len ground_truth 61717404
len pred 4994922
min length4994922


100%|██████████| 4994922/4994922 [00:18<00:00, 264790.88it/s]
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\ysouley\AppData\Local\Temp\ipykernel_23924\1795213089.py", line 110, in <lambda>
    self.compute_button = tk.Button(windows, text="Compute Metrics",command=lambda: self.compute_metrics(self.gtruth_file_entry.get(),self.pred_file_entry.get(),self.json_file_entry.get()))
                                                                                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ysouley\AppData\Local\Temp\ipykernel_23924\1795213089.py", line 243, in compute_metrics
    metrics_fr = m.compute_metrics()
                 ^^^^^^^^^^^^^^^^^^^
  File "m:\#Utilisateurs\YSO\YSO_scripts\metric

len ground_truth 61717404
len pred 61717404
min length61717404


100%|██████████| 61717404/61717404 [03:52<00:00, 265413.60it/s]
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ysouley\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is mo